# Modules

In [1]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math

In [2]:
pn.__version__

'0.13.1'

# Classes

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Functions

In [4]:
def bm_2d(n_steps = 1000, speed = 6, s_pos = [0,0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Return:
    BM_2d_df
  """
  # Init velocity vector
  velocity = Vec2d(speed,0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
  return BM_2d_df

# Panel examples

In [5]:
pn.Column('row1','row2','row3')

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)

In [6]:
pn.Row('ColumnA', 'ColumnB', 'ColumnC')

Row
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)

# Integer values

In [7]:
Integer_A = pnw.IntInput(name='An Integer Value', value=30, step=5, start=0, end=200)

In [8]:
Integer_A

IntInput(end=200, name='An Integer Value', start=0, step=5, value=30)

In [9]:
@pn.depends(Integer_A)
def display_Int_A(Integer_A):
  return Integer_A

In [10]:
Integer_A.value

30

In [11]:
pn.Column(Integer_A, display_Int_A)

Column
    [0] IntInput(end=200, name='An Integer Value', start=0, step=5, value=30)
    [1] ParamFunction(function)

In [12]:
Integer_B = pnw.IntSlider(name='Another Integer Value', width = 400, value=50, step=10, start=0, end=200)
Integer_B

IntSlider(end=200, name='Another Integer Value', step=10, value=50, width=400)

# Radio Button

In [13]:
Radio_Button_ex = pnw.RadioBoxGroup(name='Radio Buttons', value='Default', options=['Default', 'otra', 'otra mas'], inline=True)
Radio_Button_ex

RadioBoxGroup(inline=True, name='Radio Buttons', options=['Default', 'otra', ...], value='Default')

In [14]:
@pn.depends(Radio_Button_ex)
def display_Radio_Button(Radio_Button_ex):
  return Radio_Button_ex

In [15]:
pn.Column(Radio_Button_ex, display_Radio_Button)

Column
    [0] RadioBoxGroup(inline=True, name='Radio Buttons', options=['Default', 'otra', ...], value='Default')
    [1] ParamFunction(function)

# Drop down menu

In [16]:
DD_selection = pnw.Select(name='Drop down menu', value='tres', options=['uno', 'dos', 'tres'])
DD_selection

Select(name='Drop down menu', options=['uno', 'dos', 'tres'], value='tres')

# Dataframe

In [17]:
# Widget
n_steps = pnw.IntSlider(name='Number of steps', width = 400, value=10, step=10, start=10, end=1000)

@pn.depends(n_steps)
def generate_df(n_steps):
  return bm_2d(n_steps)

pn.Row(n_steps, generate_df)

Row
    [0] IntSlider(end=1000, name='Number of steps', start=10, step=10, value=10, width=400)
    [1] ParamFunction(function)

# Trajectory

In [18]:
n_steps = pnw.IntSlider(name='Number of steps', width = 400, value=10, step=10, start=10, end=1000)
s_x_pos = pnw.IntInput(name='Starting pos X', value=10, step=10, start=-100, end=100)

@pn.depends(n_steps, s_x_pos)
def plot_traj(n_steps, s_x_pos):
  fig_traj_rw = go.Figure()

  rw_df = bm_2d(n_steps, s_x_pos)

  fig_traj_rw.add_trace(
      go.Scatter3d(
          x = rw_df.x_pos,
          y = rw_df.y_pos,
          z = rw_df.index,
          marker = dict(size=2),
          line = dict(color='red', width=2),
          mode = 'lines',
          name = f'steps = {n_steps}',
          showlegend = True
      )
  )
  return fig_traj_rw

pn.Column(pn.Row(n_steps, s_x_pos), plot_traj)

Column
    [0] Row
        [0] IntSlider(end=1000, name='Number of steps', start=10, step=10, value=10, width=400)
        [1] IntInput(end=100, name='Starting pos X', start=-100, step=10, value=10)
    [1] ParamFunction(function)